# Train gradient boosting model using Bayesian optimization to find best hyperparameters

# 1. Imports

## 1.1 Packages

In [14]:
import sys

import pandas as pd


## 1.2 Options

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
sys.path.append('../src')

# from velib_prediction.pipelines.train_model.mlflow import (  # noqa: E402
#     create_mlflow_experiment,
# )
from velib_prediction.pipelines.train_model.nodes import (  # noqa: E402
    add_lags_sma,
    get_split_train_val_cv,
    split_train_valid,
    # train_model_bayesian_opti,
    train_model_mlflow,
)


In [17]:
lags_to_try = [1,]

In [18]:
feat_date = "duedate"

## 1.3 Datasets

In [19]:
df_training = pd.read_parquet("../data/04_feature/df_feat_train.parquet")
df_training.sample(2)

,idx,stationcode,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
7,110251729565473,11025,1,43,5,37,30,7,1,1,2024-10-22 02:51:13+00:00,75056,2024,10,22,1,0
8,170411729893708,17041,1,36,21,14,12,2,1,1,2024-10-25 22:01:48+00:00,75056,2024,10,25,4,0


In [20]:
df_training.rename(columns={"numbikesavailable": "target"}, inplace=True)

In [21]:
df_training.tail()

,idx,stationcode,is_installed,capacity,numdocksavailable,target,mechanical,ebike,is_renting,is_returning,duedate,code_insee_commune,duedate_year,duedate_month,duedate_day,duedate_weekday,duedate_weekend
5,170411729666463,17041,1,36,28,8,8,0,1,1,2024-10-23 06:54:23+00:00,75056,2024,10,23,2,0
6,150471729666488,15047,1,52,48,2,2,0,1,1,2024-10-23 06:54:48+00:00,75056,2024,10,23,2,0
7,121091729666278,12109,1,30,22,8,6,2,1,1,2024-10-23 06:51:18+00:00,75056,2024,10,23,2,0
8,141081729666376,14108,1,21,17,3,2,1,1,1,2024-10-23 06:52:56+00:00,75056,2024,10,23,2,0
9,161381729666193,16138,1,30,15,14,10,4,1,1,2024-10-23 06:49:53+00:00,75056,2024,10,23,2,0


# 2. Prepare datasets

In [22]:
# Add lags defined
df_training = add_lags_sma(df_training, lags_to_try, feat_id='stationcode', feat_date=feat_date, feat_target="target", n_shift=5)

In [23]:
df_training.sort_values(by="duedate", inplace=True)

In [24]:
# df_training.drop(columns="duedate", inplace=True)
# list_df = get_split_train_val_cv(df_training, n_splits=2)

In [25]:
def _filter_last_hours(group, n_hours: int=5):
    """
    """
    last_hours = group["duedate"].max() - pd.Timedelta(hours=n_hours)
    return group[group["duedate"] >= last_hours]


df = df_training.sort_values(["stationcode", "duedate"], ascending=[True, True])
df.groupby('stationcode').apply(_filter_last_hours, n_hours=5).reset_index(drop=True)["idx"].values

<StringArray>
['100131730315685', '110251730746766', '111041730739229', '111041730746799',
 '121091730739548', '130071730721289', '130071730725164', '131011730739460',
 '131181730315330', '140141730746680', '141081730649363', '141111730721327',
 '141111730725161', '150471730724617', '150681730156632', '152021730725089',
 '161071730720977', '161381730710697', '170251730533538', '170261730657809',
 '170261730667239', '170411730710650', '170441730739499', '190271730436581',
 '190331730724706', '190331730739553', '201431730616370',  '20171730315605',
  '20221730725147',  '20221730739586', '210101730721502', '210101730725112',
  '50161730739248',  '50161730746627',  '51101730746760',  '60031730746806',
  '60211730739513',  '61081730739563',  '61081730746587',  '70021730742869',
  '70031730746756',  '80261730746814',  '80501730710741',  '80501730725157',
  '81031730315677',  '90201730739502']
Length: 46, dtype: string

In [ ]:
df

In [27]:
df["idx"]

6    100131730315685
7    110251729352699
6    110251729356310
5    110251729363427
6    110251729377872
          ...       
0     90201730657755
0     90201730657755
0     90201730665294
0     90201730667493
0     90201730739502
Name: idx, Length: 3620, dtype: string

In [13]:
# df_train, df_valid = list_df[0][0], list_df[0][1]

df_train, df_valif = split_train_valid(df_training, n_hours=5)

KeyError: "None of [Index(['100131730315685', '110251730746766', '111041730739229',\n       '111041730746799', '121091730739548', '130071730721289',\n       '130071730725164', '131011730739460', '131181730315330',\n       '140141730746680', '141081730649363', '141111730721327',\n       '141111730725161', '150471730724617', '150681730156632',\n       '152021730725089', '161071730720977', '161381730710697',\n       '170251730533538', '170261730657809', '170261730667239',\n       '170411730710650', '170441730739499', '190271730436581',\n       '190331730724706', '190331730739553', '201431730616370',\n       '20171730315605', '20221730725147', '20221730739586', '210101730721502',\n       '210101730725112', '50161730739248', '50161730746627', '51101730746760',\n       '60031730746806', '60211730739513', '61081730739563', '61081730746587',\n       '70021730742869', '70031730746756', '80261730746814', '80501730710741',\n       '80501730725157', '81031730315677', '90201730739502'],\n      dtyp

In [27]:
feat_cat = [
    "is_installed",
    "is_renting",
    "is_returning",
    "code_insee_commune",
    "duedate_weekend",
]

# 3. Train model

In [28]:
# experiment_id = create_mlflow_experiment(
#     experiment_folder_path="../data/06_models/mlruns",
#     experiment_name="velib_prediction"
# )
# experiment_id

experiment_id = '587291553688351204'

In [29]:
# search_params = {
#     "iterations": (100, 500),
#     "depth": (2, 7),
# }

params = {
    "iterations": 100,
    "depth": 4
}

In [30]:
# train_model_bayesian_opti(
#     run_name="Test_catboost",
#     experiment_id=experiment_id,
#     search_params=search_params,
#     df_train=df_train,
#     df_valid=df_valid,
#     feat_cat=feat_cat,
#     n_trials=10
# )

model, rmse_train, rmse_valid = train_model_mlflow(
    experiment_id=experiment_id,
    parent_run_id=None,
    df_train=df_train,
    df_valid=df_valid,
    feat_cat=feat_cat,
    **params
)

print("RMSE train:", rmse_train)
print("RMSE valid:", rmse_valid)

/Users/benjaminwallyn/Git/velib-prediction/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/26 22:05:49 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: MlflowException("Failed to enforce schema of data '                  idx stationcode  is_

RMSE train: 0.2992554793036776
RMSE valid: 0.7423017662117376
